In [132]:
import torch
import torch.nn as nn
import matplotlib
from torch.autograd import Variable
matplotlib.use('TkAgg')    # My buggy OSX 10.13.6 requires this
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
import random
N = 2000
batch_size = 10
epochs = N//batch_size
hidden_size = 5
output_size = 1
lr = 0.02

In [133]:
def return_xor(N):
    tmp_x = []
    tmp_y = []
    for i in range(N):
        a = (random.randint(0, 1) == 1)
        b = (random.randint(0, 1) == 1)
        if (a and not b) or (not a and b):
            q = True
        else:
            q = False
        input_features = (a, b)
        output_class = q
        tmp_x.append(input_features)
        tmp_y.append(output_class)
    return tmp_x, tmp_y

In [134]:
# Training set
x, y = return_xor(N)
x = torch.tensor(x, dtype=torch.float, requires_grad=True)
y = torch.tensor(y, dtype=torch.float, requires_grad=True).view(-1 ,1)
# Test dataset
x_test, y_test = return_xor(100)
x_test = torch.tensor(x_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)
print(y.shape)

torch.Size([2000, 1])


In [135]:
class MyDataset(Dataset):
    """Define my own `Dataset` in order to use `Variable` with `autograd`"""
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return len(self.x)


In [136]:
dataset = MyDataset(x,y)
test_dataset = MyDataset(x_test, y_test)

In [137]:
print(dataset.x.shape)
print(dataset.y.shape)

torch.Size([2000, 2])
torch.Size([2000, 1])


In [138]:
# Make data iterable by loading to a loader. Shuffle, batch_size kwargs put them here in order to remind I myself
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [139]:
print(f"They are {len(train_loader)} batches in the dataset")
shown = 0
for (x, y) in train_loader:
    if shown == 1:
        break
    print(f"{x.shape} {x.dtype}")
    print(f"{y.shape} {y.dtype}")
    shown += 1

They are 200 batches in the dataset
torch.Size([10, 2]) torch.float32
torch.Size([10, 1]) torch.float32


In [145]:
class MyModel(nn.Module):
    """
    Binary classification
    """
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        # Apply He initialization to first layer because of ReLU
        torch.nn.init.normal_(self.fc1.weight, mean=0, std=1) # sqrt(2/2) is 1 See the Fawaz lecture P27
        # Apply Xavier to second layer because of Sigmoid
        torch.nn.init.xavier_normal_(self.fc2.weight)
        
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, out):
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
        

In [146]:
# Create my network
net = MyModel(dataset.x.shape[1], hidden_size, output_size)

# # https://courses.cs.washington.edu/courses/cse446/18wi/sections/section8/XOR-Pytorch.html
# def weights_init(model):
#     for m in model.modules():
#         if isinstance(m, nn.Linear):
#             # initialize the weight tensor, here we use a normal distribution
#             m.weight.data.normal_(0, 1)

# weights_init(net)

CUDA = torch.cuda.is_available()
if CUDA:
    net = net.cuda()
# In Cross Entropy Loss: Input shape (N,C) and labels shape(N)
# In Binary Cross Entropy: Input and output should have the same shape
# size_average = True ->losses are averaged over observations for each minibatch -
# Got warning use `reduction='elementwise_mean'`
# criterion = torch.nn.BCELoss(reduction='elementwise_mean')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [147]:
# Train the network
correct_train = 0
total_train = 0
for epoch in range(epochs):
    for i, (batches, labels) in enumerate(train_loader):
        batcesh = Variable(batches.float())
        labels = Variable(labels.float())

        optimizer.zero_grad()
        output = net(batches)                                      # Forward pass

        loss = criterion(output, labels.view(batch_size, 1))            
        loss.backward()
        optimizer.step()

        total_train += labels.size(0)
        correct_train += (output.round().long() == labels.long()).sum()
        if i % 100 == 0:
            print(f"""
                Epoch {epoch+1}/{epochs}, 
                Iteration {i+1}/{len(dataset)//batch_size}, 
                Training Loss: {loss.item()},
                Training Accuracy: {100*correct_train/total_train}
              """)

        


                Epoch 1/200, 
                Iteration 1/200, 
                Training Loss: 0.2293422967195511,
                Training Accuracy: 60
              

                Epoch 1/200, 
                Iteration 101/200, 
                Training Loss: 0.02658955380320549,
                Training Accuracy: 82
              

                Epoch 2/200, 
                Iteration 1/200, 
                Training Loss: 0.0024163220077753067,
                Training Accuracy: 91
              

                Epoch 2/200, 
                Iteration 101/200, 
                Training Loss: 0.0007453538128174841,
                Training Accuracy: 94
              

                Epoch 3/200, 
                Iteration 1/200, 
                Training Loss: 0.0004876851453445852,
                Training Accuracy: 95
              

                Epoch 3/200, 
                Iteration 101/200, 
                Training Loss: 0.00037203452666290104,
                Tra


                Epoch 25/200, 
                Iteration 101/200, 
                Training Loss: 2.270136292281677e-06,
                Training Accuracy: 99
              

                Epoch 26/200, 
                Iteration 1/200, 
                Training Loss: 2.153865580112324e-06,
                Training Accuracy: 99
              

                Epoch 26/200, 
                Iteration 101/200, 
                Training Loss: 1.8180945744461496e-06,
                Training Accuracy: 99
              

                Epoch 27/200, 
                Iteration 1/200, 
                Training Loss: 1.861565806393628e-06,
                Training Accuracy: 99
              

                Epoch 27/200, 
                Iteration 101/200, 
                Training Loss: 1.74948877429415e-06,
                Training Accuracy: 99
              

                Epoch 28/200, 
                Iteration 1/200, 
                Training Loss: 1.4574181932403008e-06,
        


                Epoch 49/200, 
                Iteration 101/200, 
                Training Loss: 1.4840958328932174e-07,
                Training Accuracy: 99
              

                Epoch 50/200, 
                Iteration 1/200, 
                Training Loss: 1.299706866575434e-07,
                Training Accuracy: 99
              

                Epoch 50/200, 
                Iteration 101/200, 
                Training Loss: 1.15538654199554e-07,
                Training Accuracy: 99
              

                Epoch 51/200, 
                Iteration 1/200, 
                Training Loss: 1.27070734379231e-07,
                Training Accuracy: 99
              

                Epoch 51/200, 
                Iteration 101/200, 
                Training Loss: 1.1551122724995366e-07,
                Training Accuracy: 99
              

                Epoch 52/200, 
                Iteration 1/200, 
                Training Loss: 8.73665939593593e-08,
          


                Epoch 73/200, 
                Iteration 101/200, 
                Training Loss: 1.0747703171887224e-08,
                Training Accuracy: 99
              

                Epoch 74/200, 
                Iteration 1/200, 
                Training Loss: 1.0683188556015466e-08,
                Training Accuracy: 99
              

                Epoch 74/200, 
                Iteration 101/200, 
                Training Loss: 9.276759804777157e-09,
                Training Accuracy: 99
              

                Epoch 75/200, 
                Iteration 1/200, 
                Training Loss: 9.240950227251687e-09,
                Training Accuracy: 99
              

                Epoch 75/200, 
                Iteration 101/200, 
                Training Loss: 8.964281761336679e-09,
                Training Accuracy: 99
              

                Epoch 76/200, 
                Iteration 1/200, 
                Training Loss: 9.219346175370902e-09,
       


                Epoch 98/200, 
                Iteration 101/200, 
                Training Loss: 8.334430923184755e-10,
                Training Accuracy: 99
              

                Epoch 99/200, 
                Iteration 1/200, 
                Training Loss: 7.138887259117155e-10,
                Training Accuracy: 99
              

                Epoch 99/200, 
                Iteration 101/200, 
                Training Loss: 6.899052995557042e-10,
                Training Accuracy: 99
              

                Epoch 100/200, 
                Iteration 1/200, 
                Training Loss: 5.995646201739646e-10,
                Training Accuracy: 99
              

                Epoch 100/200, 
                Iteration 101/200, 
                Training Loss: 6.518962036849985e-10,
                Training Accuracy: 99
              

                Epoch 101/200, 
                Iteration 1/200, 
                Training Loss: 6.122597984159484e-10,
      


                Epoch 123/200, 
                Iteration 1/200, 
                Training Loss: 6.959118697968947e-11,
                Training Accuracy: 99
              

                Epoch 123/200, 
                Iteration 101/200, 
                Training Loss: 6.86150164463939e-11,
                Training Accuracy: 99
              

                Epoch 124/200, 
                Iteration 1/200, 
                Training Loss: 5.622808815775038e-11,
                Training Accuracy: 99
              

                Epoch 124/200, 
                Iteration 101/200, 
                Training Loss: 6.670856922408319e-11,
                Training Accuracy: 99
              

                Epoch 125/200, 
                Iteration 1/200, 
                Training Loss: 6.394702822820619e-11,
                Training Accuracy: 99
              

                Epoch 125/200, 
                Iteration 101/200, 
                Training Loss: 6.208349806469116e-11,
    


                Epoch 147/200, 
                Iteration 1/200, 
                Training Loss: 1.1817239894962306e-11,
                Training Accuracy: 99
              

                Epoch 147/200, 
                Iteration 101/200, 
                Training Loss: 1.0872385457216804e-11,
                Training Accuracy: 99
              

                Epoch 148/200, 
                Iteration 1/200, 
                Training Loss: 9.469504173853505e-12,
                Training Accuracy: 99
              

                Epoch 148/200, 
                Iteration 101/200, 
                Training Loss: 1.0544967676828776e-11,
                Training Accuracy: 99
              

                Epoch 149/200, 
                Iteration 1/200, 
                Training Loss: 1.1002741252819082e-11,
                Training Accuracy: 99
              

                Epoch 149/200, 
                Iteration 101/200, 
                Training Loss: 1.0119708890310442e-11


                Epoch 172/200, 
                Iteration 1/200, 
                Training Loss: 3.823160971833106e-12,
                Training Accuracy: 99
              

                Epoch 172/200, 
                Iteration 101/200, 
                Training Loss: 4.333873537820665e-12,
                Training Accuracy: 99
              

                Epoch 173/200, 
                Iteration 1/200, 
                Training Loss: 3.802233267818922e-12,
                Training Accuracy: 99
              

                Epoch 173/200, 
                Iteration 101/200, 
                Training Loss: 3.5845501913933653e-12,
                Training Accuracy: 99
              

                Epoch 174/200, 
                Iteration 1/200, 
                Training Loss: 3.714197352455706e-12,
                Training Accuracy: 99
              

                Epoch 174/200, 
                Iteration 101/200, 
                Training Loss: 3.826405338414052e-12,
  


                Epoch 196/200, 
                Iteration 1/200, 
                Training Loss: 2.3264456767285724e-12,
                Training Accuracy: 99
              

                Epoch 196/200, 
                Iteration 101/200, 
                Training Loss: 2.09447151462705e-12,
                Training Accuracy: 99
              

                Epoch 197/200, 
                Iteration 1/200, 
                Training Loss: 2.266273757198234e-12,
                Training Accuracy: 99
              

                Epoch 197/200, 
                Iteration 101/200, 
                Training Loss: 2.3543465354353144e-12,
                Training Accuracy: 99
              

                Epoch 198/200, 
                Iteration 1/200, 
                Training Loss: 2.3182668892202107e-12,
                Training Accuracy: 99
              

                Epoch 198/200, 
                Iteration 101/200, 
                Training Loss: 2.5005093481911267e-12,


In [148]:
# Test the network
# WIP
correct_answer = 0
for i, (batches, labels) in enumerate(test_dataset):
    batches = Variable(batches.float())
    labels = Variable(labels.float())

    output = net(batches)                       
    assertion = output.round().item() == labels.item()
    print(output.round().item(), labels.item(), assertion)
    correct_answer += assertion
print(f"Testing Accuracy: {100 * correct_answer/test_dataset.x.shape[0]}%")


0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
0.0 0.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
0.0 0.0 True
1.0 1.0 True
0.0 0.0 True
0.0 0.0 True